In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import datetime

In [2]:
# Read Global Temperatures by Country file and store into Pandas data frame
land_temp_by_country_csv = "../../ProjectOne_datasets/GlobalLandTemperaturesByCountry.csv"
land_temp_by_country_df = pd.read_csv(land_temp_by_country_csv)
land_temp_by_country_df.tail()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
577457,2013-05-01,19.059,1.022,Zimbabwe
577458,2013-06-01,17.613,0.473,Zimbabwe
577459,2013-07-01,17.000,0.453,Zimbabwe
577460,2013-08-01,19.759,0.717,Zimbabwe
577461,2013-09-01,NaN,NaN,Zimbabwe


In [3]:
# Read Population Growth by Country file and store into Pandas data frame
population_growth_country_csv = "../../ProjectOne_datasets/population-by-country.csv"
population_growth_country_df = pd.read_csv(population_growth_country_csv)
population_growth_country_df.tail()

#Rename Column for World Population data
population_growth_country_df = population_growth_country_df.rename(columns={"Population by Country (Clio Infra (2016))":"Population",
                                                                           "Entity":"Country"})
population_growth_country_df.head()

,Country,Code,Year,Population
0,Afghanistan,AFG,1500,2000000.0
1,Afghanistan,AFG,1600,2500000.0
2,Afghanistan,AFG,1700,2500000.0
3,Afghanistan,AFG,1800,3280000.0
4,Afghanistan,AFG,1820,3280000.0


In [4]:
#Load CO2 Emissions Growth by Country file and store into Pandas data frame
co2_country_csv = "../../ProjectOne_datasets/annual-co-emissions-by-region.csv"
co2_country_df = pd.read_csv(co2_country_csv)
co2_country_df.tail()

#Rename Column 
co2_country_df = co2_country_df.rename(columns={"Entity":"Country"})
co2_country_df.head()

,Country,Code,Year,Annual CO2 emissions
0,Afghanistan,AFG,1750,0.0
1,Afghanistan,AFG,1751,0.0
2,Afghanistan,AFG,1752,0.0
3,Afghanistan,AFG,1753,0.0
4,Afghanistan,AFG,1754,0.0


In [5]:
# Cleanup Land Temperature Datasets
########################################

#Drop AverageTemperatureUncertainty
land_temp_by_country_df = land_temp_by_country_df.drop(["AverageTemperatureUncertainty"], axis=1)

#Remove rows with null temperatures
land_temp_by_country_df = land_temp_by_country_df[land_temp_by_country_df["AverageTemperature"].notna()]

print(f"Country Data Size: {len(land_temp_by_country_df)}")

Country Data Size: 544811


In [6]:
#Retrieve rows for the last century
cent_land_temp_by_country_df = land_temp_by_country_df.loc[land_temp_by_country_df["dt"] >= "1900-01-01"]
clean_co2_df=co2_country_df.loc[co2_country_df['Year'] >= 1900]

In [14]:
#For CO2 Emissions, add bins and cut based on group names
bins = [1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010]
group_names = ["1900", "1910", "1920", "1930", "1940", "1950", "1960", "1970", "1980", "1990", "2000"]

clean_co2_df["Decade"] = pd.cut(clean_co2_df["Year"], bins, labels=group_names, include_lowest=True)

decade_co2 = clean_co2_df.groupby(['Decade', 'Country']).agg({'Annual CO2 emissions': ['mean']})
decade_co2


<ipython-input-14-ece354d1fadf>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_co2_df["Decade"] = pd.cut(clean_co2_df["Year"], bins, labels=group_names, include_lowest=True)


Annual CO2 emissions
                                                 mean
Decade Country                                       
1900   Afghanistan                       0.000000e+00
       Africa                            9.998390e+06
       Albania                           0.000000e+00
       Algeria                           0.000000e+00
       Andorra                           0.000000e+00
...                                               ...
2000   Wallis and Futuna Islands         2.601440e+04
       World                             2.945115e+10
       Yemen                             1.983259e+07
       Zambia                            2.144057e+06
       Zimbabwe                          9.640064e+06

[2574 rows x 1 columns]

In [15]:
global countries_temperatures_df 
columns = ["Decade", "Land Temperature", "Country"]
countries_temperatures_df = pd.DataFrame(columns = columns)

def get_surface_temps_by_country(country):
    global countries_temperatures_df 
    temp_df = cent_land_temp_by_country_df.loc[land_temp_by_country_df["Country"]==country]
    temp_df = temp_df.groupby((pd.DatetimeIndex(temp_df["dt"]).year//10)*10).mean()
    temp_df["Country"] = country

    #Reset index, and rename columns appropriately
    temp_df.reset_index(inplace=True)
    temp_df = temp_df.rename(columns = {'dt':'Decade'})
    temp_df = temp_df.rename(columns = {'AverageTemperature':'Land Temperature'})
    temp_df = temp_df.loc[temp_df["Decade"] != 2010]
    temp_df
    countries_temperatures_df = pd.concat([countries_temperatures_df, temp_df], axis=0)

In [16]:
global countries_population_df 
columns = ["Decade", "Population", "Country"]
countries_population_df = pd.DataFrame(columns = columns)

def get_population_by_country(country):
    global countries_population_df 
    
    temp_df = population_growth_country_df.loc[population_growth_country_df["Country"] == country]
    temp_df = temp_df.loc[temp_df["Year"] >= 1900]
    temp_df = temp_df.rename(columns = {'Year':'Decade'})
    temp_df = temp_df.drop(["Code"], axis=1)
    countries_population_df = pd.concat([countries_population_df, temp_df], axis=0)

In [17]:
list_of_countries = ["United States", "China", "Brazil", "Denmark", "India"]

for country in list_of_countries:
    get_surface_temps_by_country(country)
    get_population_by_country(country)

In [18]:
idx = pd.IndexSlice
countries_CO2_df=decade_co2.loc[idx[:,list_of_countries], :]

print(f"Temperature lenght: {len(countries_temperatures_df)}")
print(f"Temperature lenght: {len(countries_population_df)}")     

countries_CO2_df.reset_index(drop=True)
countries_CO2_df.columns.droplevel()
countries_CO2_df

Temperature lenght: 55
Temperature lenght: 55


,,Annual CO2 emissions
,,mean
Decade,Country,
1900,United States,9.697364e+08
1910,United States,1.493924e+09
1920,United States,1.753823e+09
1930,United States,1.556552e+09
1940,United States,2.330513e+09
1950,United States,2.710035e+09
1960,United States,3.504613e+09
1970,United States,4.650222e+09


In [20]:
#Final cleansed and merged datasets
final_population_land_temps_df = pd.merge(countries_temperatures_df, countries_population_df, on=["Decade", "Country"])

print(f"Merged Data Set lenght: {len(final_population_land_temps_df)}") 
final_population_land_temps_df



Merged Data Set lenght: 55


,Decade,Land Temperature,Country,Population
0,1900,8.448225,United States,7.639100e+07
1,1910,8.460250,United States,9.276700e+07
2,1920,8.755250,United States,1.068810e+08
3,1930,9.086992,United States,1.236680e+08
4,1940,8.924092,United States,1.326370e+08
5,1950,8.913233,United States,1.522710e+08
6,1960,8.721917,United States,1.806710e+08
7,1970,8.696925,United States,2.050520e+08
8,1980,9.104308,United States,2.277265e+08
9,1990,9.317558,United States,2.501319e+08


In [22]:
#Reorganize the column names for ease of redability
column_names = ["Decade", "Country", "Land Temperature", "Population"]
final_population_land_temps_df = final_population_land_temps_df.reindex(columns=column_names)
final_population_land_temps_df.set_index(["Decade", "Country"])
final_population_land_temps_df




,Decade,Country,Land Temperature,Population
0,1900,United States,8.448225,7.639100e+07
1,1910,United States,8.460250,9.276700e+07
2,1920,United States,8.755250,1.068810e+08
3,1930,United States,9.086992,1.236680e+08
4,1940,United States,8.924092,1.326370e+08
5,1950,United States,8.913233,1.522710e+08
6,1960,United States,8.721917,1.806710e+08
7,1970,United States,8.696925,2.050520e+08
8,1980,United States,9.104308,2.277265e+08
9,1990,United States,9.317558,2.501319e+08


In [27]:
# Push the remade DataFrame to a new CSV file
final_population_land_temps_df.to_csv("../Output/Population_SurfaceTemps_By_Country.csv",
                  encoding="utf-8", index=False, header=True)

countries_CO2_df.to_csv("../Output/CO2Emissions_By_Country.csv",
                  encoding="utf-8", index=True, header=True)